In [ ]:
import os

## dot stimulus

see https://github.com/laurentperrinet/Khoei_2017_PLoSCB/blob/master/scripts/MotionParticlesFLE.py#L1946



In [ ]:
mwidth = 1024
fps = 50
simtime = 350 # ms
time_bins = 350
dt = simtime/time_bins
N_pop = 128**2


In [3]:
import numpy as np

def generate_dot(N_X, N_Y, N_frame,
                X_0=-1., Y_0=0.0, #initial position
                V_X=1., V_Y=0., #target speed
                dot_size=.05,
                blank_duration=0., blank_start=0.,
                flash_duration=0., flash_start=0., #flashing=False,
                width=2.,
                im_noise=0.05, #std of the background noise in images
                im_contrast=1.,
                NoisyTrajectory=False, traj_Xnoise=0, traj_Vnoise=0, reversal=False,
                hard=False, second_order=False, f=8, texture=False, sf_0=0.15,
                pink_noise=False, do_clip=True,
                **kwargs):

    """

    >> pylab.imshow(concatenate((image[16,:,:],image[16,:,:]), axis=-1))

    """
    r_x = width / 2.
    r_y = r_x * N_Y / N_X
    x, y, t = np.mgrid[r_x*(-1+1./(N_X)):r_x*(1-1./(N_X)):1j*N_X,
                    r_y*(-1+1./(N_Y)):r_y*(1-1./(N_Y)):1j*N_Y,
                    0:(1-1./(N_frame+1)):1j*N_frame]

    if NoisyTrajectory:
       V_Y +=  traj_Vnoise/ np.float(N_frame) * np.random.randn(1, N_frame)
       V_X +=  traj_Vnoise/ np.float(N_frame) * np.random.randn(1, N_frame)

    x_ = np.amin([np.abs((x - X_0) - V_X*t*width), width - np.abs((x - X_0) - V_X*t*width)], axis=0)
    y_ = np.amin([np.abs((y - Y_0)- (V_Y )*t*width), width * N_Y / N_X - np.abs((y - Y_0) - V_Y*t*width)], axis=0)

    tube = np.exp(- (x_**2 + y_**2) /2. / dot_size**2) # size is relative to the width of the torus

    if hard : tube = (tube > np.exp(-1./2)) * 1.

    if texture:
        from experiment_cloud import generate as cloud
        texture = cloud(N_X, N_Y, N_frame, V_X=V_X, V_Y=V_Y, sf_0=sf_0, noise=0)
        texture /= np.abs(texture).max()
        tube *= texture
        #np.random.rand(N_X, N_Y, N_frame)

    if second_order:
        from experiment_fullgrating import generate as fullgrating
        tube *= fullgrating(N_X=N_X, N_Y=N_Y, N_frame=N_frame, width=width,
                            V_X=0., noise=0., f=f)

    tube /= tube.max()
    tube *= im_contrast

    # trimming
    if blank_duration>0.:
        N_start = np.floor(blank_start * N_frame)
        N_blank = np.floor(blank_duration * N_frame)
        tube[:, :, N_start:N_start + N_blank] = 0.

    if flash_duration>0.:
        N_start = int(flash_start * N_frame)
        N_flash = int(flash_duration * N_frame)
        # to have stationary flash, set speed to zero
        tube[:, :, :N_start] = 0.
        tube[:, :, (N_start + N_flash):] = 0.

    if reversal:
        # mirroring the second half period
        tube[:, :, N_frame//2:] = tube[::-1, :, N_frame//2:]

    # adding noise
    if pink_noise:
        from MotionClouds import get_grids, envelope_color, random_cloud, rectif
        fx, fy, ft = get_grids(N_X, N_Y, N_frame)
        envelope = envelope_color(fx, fy, ft, alpha=1) #V_X=V_X*N_Y/N_frame, V_Y=V_Y*N_X/N_frame, sf_0=sf_0, B_V=B_V, B_sf=B_sf, B_theta=B_theta)
        noise_image = 2*rectif(random_cloud(envelope))-1
#         print(noise_image.min(), noise_image.max())
        tube += im_noise * noise_image #np.random.randn(N_X, N_Y, N_frame)
    else:
        tube += im_noise * np.random.randn(N_X, N_Y, N_frame)

    if do_clip:
        tube = np.maximum( -1, np.minimum( 1, tube ) )
        
    return tube

## movie generation

see https://github.com/laurentperrinet/Khoei_2017_PLoSCB/blob/master/scripts/MotionParticlesFLE.py#L1946



In [4]:
def rectify(movie):
    return (movie + 1) / 2

def minmax(movie):
    print(f'{movie.min()=} - {movie.max()=} ')

In [5]:
output_dir = 'output'

In [6]:
%mkdir -p {output_dir}

In [7]:
import imageio
def generate_images(movie, label='image', output_dir=output_dir, verbose=False):
    fnames = []
    N_frame = movie.shape[2]
    for i_image in range(N_frame):
        fname = os.path.join(output_dir, f'{label}_{i_image:05d}.png')
        if verbose: print('fname=', fname)
        frame = movie[:, :, i_image]
        frame = frame[:, :, None] * np.ones((1, 1, 3))
        frame = (frame*255).astype(np.uint8)
        imageio.imsave(fname, frame, dpi=(36, 36))
        fnames.append(fname)
    return fnames

In [8]:
from IPython.display import Video, HTML

def make_gif(gifname, fnames, fps):
    import imageio

    with imageio.get_writer(gifname, mode='I', fps=fps) as writer:
        for fname in fnames:
            writer.append_data(imageio.imread(fname))

    from pygifsicle import optimize
    optimize(str(gifname))
    return gifname

import moviepy.editor as mpy

import moviepy.video.io.ImageSequenceClip
def make_mp4(moviename, fnames, fps):
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(fnames, fps=fps)
    clip.write_videofile(moviename)
    return moviename

def cleanup(fnames):
    for fname in fnames: os.remove(fname)
    

def show(filename, mwidth=mwidth):
    return mpy.ipython_display(filename, fps=fps, mwidth=mwidth, autoplay=True, loop=True)

def make_movie(movie, label, fps=fps):
    fnames = generate_images(movie, label='image', output_dir=output_dir, verbose=False)
    moviename = f'{output_dir}/{label}.mp4'
    moviename = make_mp4(moviename, fnames, fps)
    cleanup(fnames)
    return moviename


In [9]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,pyNN,neo,moviepy,imageio  -r -g -b

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0
moviepy   : 1.0.3
imageio   : 2.9.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-53-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: 48979300bebb405fe2ff6bd586076952fed3996a

Git repo: https://github.com/SpikeAI/2020-11_brainhack_Project7

Git branch: main

